In [70]:
import pandas as pd
import sqlite3 as db
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler, OrdinalEncoder, FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
# python
import pickle
from functools import partial
# basics
import scipy.stats as stats
# graphing
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.metrics import plot_confusion_matrix
# feature selection
from sklearn.feature_selection import SelectKBest, SelectPercentile, chi2
# model selection
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sklearn.metrics import (r2_score, mean_squared_error, accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, roc_curve, precision_recall_curve, make_scorer,
                             confusion_matrix, multilabel_confusion_matrix, ConfusionMatrixDisplay)
# models
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LogisticRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.dummy import DummyClassifier
import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor
from imblearn.over_sampling import SMOTE
from collections import Counter
from datetime import datetime
from functions import training_dataset, PrepareWordLists, selectdata, readdata, cleandatastructure, buildwordsindex, trainingstructuredata, getlistofwords
#import importlib as implib
import gc

import unidecode
import string
import re
import json
from names_dataset import NameDataset

## Import Excel File



In [4]:
#ISM_SourceData = pd.read_excel('Data/Tickets 2021-2022 (V02).xlsx', sheet_name="Squirrel SQL Export")
#xx= ISM_SourceData.loc[6]['LONGDESCRIPTION']
#ISM_SourceData.info()
ISM_SourceData = readdata()

In [71]:
#ISM_SourceData.info()
#with open('ISM_SourceData.dataframe', 'wb') as f:
#    pickle.dump(ISM_SourceData, f, protocol=pickle.HIGHEST_PROTOCOL)
with open('ISM_SourceData.dataframe', 'rb') as f:
    ISM_SourceData = pickle.load(f)

In [72]:
ISM_SourceData_test = ISM_SourceData[( ISM_SourceData['ACTUALSTART']>="2021-10-25") & (ISM_SourceData['ACTUALSTART']<="2021-11-25")]
ISM_SourceData_test.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 32326 entries, 1 to 428978
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   TICKETID                   32326 non-null  object        
 1   TITLE                      32325 non-null  object        
 2   LONGDESCRIPTION            32092 non-null  object        
 3   STATUS                     32326 non-null  object        
 4   IMPACT                     32326 non-null  float64       
 5   URGENCY                    32326 non-null  float64       
 6   AFFECTEDPERSON             32326 non-null  object        
 7   AffectedPersonName         32315 non-null  object        
 8   AffectedPersonEmail        30160 non-null  object        
 9   AffectedPersonCountry      32323 non-null  object        
 10  CREATIONDATE               32326 non-null  datetime64[ns]
 11  CREATEDBY                  32326 non-null  object        
 12  Cre

## Cleaning Data

In [88]:
country_list = ['Mexico']

#ISM_test = ISM_test[ISM_test['AffectedPersonCountry'] in country_list == ]
ISM_test = cleandatastructure(ISM_SourceData_test,country_list )
#ISM_test = ISM_test[ISM_test['AffectedPersonCountry'].isin(country_list)]
ISM_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14450 entries, 16 to 428978
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   TICKETID               14450 non-null  object
 1   FULLDESC               14450 non-null  object
 2   AffectedPersonCountry  14450 non-null  object
 3   OWNERGROUP             14450 non-null  object
dtypes: object(4)
memory usage: 564.5+ KB


## 

In [83]:
ISM_test['AffectedPersonCountry'].unique()

array(['United Kingdom', 'Mexico', 'United States of America', 'Jamaica',
       'Spain', 'France', 'Philippines', 'Nicaragua', 'Czech Republic',
       'Colombia', 'Costa Rica', 'Poland', 'Romania',
       'Dominican Republic', 'United Arab Emirates', 'Germany', 'Croatia',
       'Central', 'Guatemala', 'Peru', 'Panama', 'Egypt', 'Israel',
       'Switzerland', 'Puerto Rico', 'Malaysia', 'Hungary', 'Haiti',
       'China', 'Bahamas', 'Netherlands', 'El Salvador', 'Latvia'],
      dtype=object)

## Prepare Gorups Word Lists


In [75]:
## Split test data into sections for performance issue
#dataset_size = 2000


TypeError: cleandatastructure() missing 1 required positional argument: 'country_list'

In [89]:
ISM_test_set1=ISM_test[0:4999]
ISM_test_set2=ISM_test[5000:9999]
ISM_test_set3=ISM_test[10000:14449]
#ISM_test_set4=ISM_test[24000:31707]
#ISM_test_set4=ISM_test[6000:7999]
#ISM_test_set5=ISM_test[8000:9999]
#ISM_test_set6=ISM_test[10000:10999]
#ISM_test_set7=ISM_test[12000:12999]
#ISM_test_set8=ISM_test[14000:15999]
#ISM_test_set9=ISM_test[16000:17356]

In [90]:
## Deffine the data set to processes
#datasets1_2_3_4 = [ISM_test_set1,ISM_test_set2,ISM_test_set3,ISM_test_set4]
datasets1 = [ISM_test_set1]
datasets2 = [ISM_test_set2]
datasets3 = [ISM_test_set3]

In [79]:
## Crear estructura para guardar los datos
ISM_wgwords = pd.DataFrame(columns=['COUNTRY', 'OWNERGROUP', 'WORDS'])

## Run de classfier words programs
## Regresa lista de palabras para cada país y groupo

ISM_wgwords0 = PrepareWordLists(datasets1_2_3_4, ISM_wgwords)
#ISM_wgwords1 = PrepareWordLists(datasets2, ISM_wgwords0)
#ISM_wgwords2 = PrepareWordLists(datasets3, ISM_wgwords1)

## Consolidate word lists if required
ISM_wgwords_all = ISM_wgwords0

#ISM_wgwords_all = ISM_wgwords0.append(ISM_wgwords1)
#ISM_wgwords_all = ISM_wgwords_all.append(ISM_wgwords2)

## Save file on excel to analyze
#ISM_wgwords_all.to_excel(f"./Data/ISMwgwords2.xlsx")

#with open('ISM_wgwords_all', 'wb') as f:
#    pickle.dump(ISM_wgwords_all, f, protocol=pickle.HIGHEST_PROTOCOL)
#with open('ISM_wgwords_all', 'rb') as f:
#    ISM_wgwords_all = pickle.load(f)


2022-03-26 19:37:42.692784
1
2022-03-26 19:38:00.072251
501
2022-03-26 19:38:17.911061
1001
2022-03-26 19:38:40.093033
1501
2022-03-26 19:39:01.231872
2001
2022-03-26 19:39:26.350613
2501
2022-03-26 19:40:01.830806
3001
2022-03-26 19:40:45.022183
3501
2022-03-26 19:41:38.598885
4001
2022-03-26 19:42:44.879934
4501
2022-03-26 19:44:02.355471
5001
2022-03-26 19:45:35.250536
5501
2022-03-26 19:47:12.505439
6001
2022-03-26 19:48:52.017144
6501
2022-03-26 19:51:02.680249
7001
2022-03-26 19:52:52.981448
7501
2022-03-26 19:55:11.746633
1
2022-03-26 19:58:55.036165
501
2022-03-26 20:01:37.265518
1001
2022-03-26 20:04:35.274478
1501
2022-03-26 20:08:10.724320
2001
2022-03-26 20:12:37.193052
2501
2022-03-26 20:17:15.570515
3001
2022-03-26 20:22:01.854681
3501
2022-03-26 20:26:09.326225
4001
2022-03-26 20:30:56.158085
4501
2022-03-26 20:35:59.839854
5001
2022-03-26 20:41:30.717751
5501
2022-03-26 20:46:41.351977
6001
2022-03-26 20:51:47.167949
6501
2022-03-26 20:57:19.711178
7001
2022-03-26 21:04

In [87]:
## Generate a unique datasat with Index for faster search

#ISM_wgwords_uniques = buildwordsindex(ISM_wgwords_all, 2)

#9:22
#with open('ISM_wgwords_uniques', 'wb') as f:
#    pickle.dump(ISM_wgwords_uniques, f, protocol=pickle.HIGHEST_PROTOCOL)
#with open('ISM_wgwords_uniques', 'rb') as f:
#    ISM_wgwords_uniques = pickle.load(f)

#ISM_wgwords_uniques = ISM_wgwords_uniques[ISM_wgwords_uniques['COUNTRY']=='Mexico']
ISM_wgwords_uniques

,COUNTRY,OWNERGROUP,WORDS,STR_INDEX
STR_INDEX,,,,
Mexico-I-CEX-IN-ECS-O365-L3,Mexico,I-CEX-IN-ECS-O365-L3,INTERNAL,Mexico-I-CEX-IN-ECS-O365-L3
Mexico-I-CEX-MX-IT-SEC-DTC-L2,Mexico,I-CEX-MX-IT-SEC-DTC-L2,INTERNAL,Mexico-I-CEX-MX-IT-SEC-DTC-L2
Mexico-I-CEX-IN-IT-IAM-DTC-L2,Mexico,I-CEX-IN-IT-IAM-DTC-L2,INTERNAL,Mexico-I-CEX-IN-IT-IAM-DTC-L2
Mexico-I-CEX-MX-COM-LE-GSC-L1,Mexico,I-CEX-MX-COM-LE-GSC-L1,INTERNAL,Mexico-I-CEX-MX-COM-LE-GSC-L1
Mexico-I-CEX-MXBOSAP-SEC-GRC-L3,Mexico,I-CEX-MXBOSAP-SEC-GRC-L3,INTERNAL,Mexico-I-CEX-MXBOSAP-SEC-GRC-L3
...,...,...,...,...
Mexico-I-CEX-MX-IT-SER-RS-L2,Mexico,I-CEX-MX-IT-SER-RS-L2,GUIAR,Mexico-I-CEX-MX-IT-SER-RS-L2
Mexico-C-CEX-MX-IT-SERVICES-RICOH-L2,Mexico,C-CEX-MX-IT-SERVICES-RICOH-L2,BVLD,Mexico-C-CEX-MX-IT-SERVICES-RICOH-L2
Mexico-C-CEX-MX-IT-SERVICES-RICOH-L2,Mexico,C-CEX-MX-IT-SERVICES-RICOH-L2,MAGANON,Mexico-C-CEX-MX-IT-SERVICES-RICOH-L2


## Preparara Datos

In [13]:
## Preparar archivo para training
## Country / <categoríaspor grupos con el número de palabras coincidentes> / Grupo <target>


In [91]:
GroupColumnsNames =ISM_wgwords_uniques['OWNERGROUP'].unique()
TrainingData = pd.DataFrame(columns=GroupColumnsNames)
TrainingData['COUNTRY'] = 'NA'
TrainingData['OWNERGROUP'] = 'NA'
TrainingData['TICKETID'] = 'NA'

#TrainingData.info()
TrainingData.COUNTRY.datatype = object
TrainingData.OWNERGROUP.datatype = object
TrainingData.TICKETID.datatype = object
#for column in TrainingData.columns:
#    print(column)

trainingstructuredata(ISM_wgwords_uniques)

,I-CEX-IN-ECS-O365-L3,I-CEX-MX-IT-SEC-DTC-L2,I-CEX-IN-IT-IAM-DTC-L2,I-CEX-MX-COM-LE-GSC-L1,I-CEX-MXBOSAP-SEC-GRC-L3,I-CEX-US-IBMSECURITYUSA-L3,I-CEX-EMEAA-BOCOM-GSC-L1,I-CEX-GLOBAL-IT-CNX-L2,I-CEX-GLOBAL-IT-LNT-GSC-L1,I-CEX-GLOBAL-IT-CNX-MON-L2,...,I-CEX-US-DSS-WPB-L2,I-CEX-MX-HR-SER-PR-L2,I-CEX-MX-CONTACTCENTER-GV-L2,C-CEX-MX-CONTACTCENTER-GV-L2,I-CEX-GLOBAL-IT-MSSSOC-INTQRAD-DTC-L2,I-CEX-MX-IT-SEC-CAB,I-CEX-MX-IT-SCCM-APP-L2,COUNTRY,OWNERGROUP,TICKETID


In [92]:
#ISM_Training_1 = training_dataset(ISM_test_set1, TrainingData, ISM_wgwords_uniques)
square_values = True

ISM_Training_1 = training_dataset(ISM_test_set1, TrainingData, ISM_wgwords_uniques, square_values)
ISM_Training_2 = training_dataset(ISM_test_set2, TrainingData, ISM_wgwords_uniques, square_values)
ISM_Training_3 = training_dataset(ISM_test_set3, TrainingData, ISM_wgwords_uniques, square_values)
#ISM_Training_4 = training_dataset(ISM_test_set4, TrainingData, ISM_wgwords_uniques, square_values)

2022-03-27 09:27:24.452853
1
2022-03-27 09:27:24.777548
2
2022-03-27 09:27:25.376536
4
2022-03-27 09:27:26.454308
8
2022-03-27 09:27:28.671995
16
2022-03-27 09:27:33.096043
32
2022-03-27 09:27:41.819707
64
2022-03-27 09:27:59.645547
128
2022-03-27 09:28:36.015793
256
2022-03-27 09:29:51.231013
512
2022-03-27 09:32:13.612047
1024
2022-03-27 09:37:04.595650
2048
2022-03-27 09:46:44.298288
4096
Finished:  2022-03-27 09:51:02.708912  Errors: 804839
2022-03-27 09:51:02.724537
1
2022-03-27 09:51:02.993879
2
2022-03-27 09:51:03.549207
4
2022-03-27 09:51:04.627850
8
2022-03-27 09:51:06.860696
16
2022-03-27 09:51:11.305877
32
2022-03-27 09:51:19.974271
64
2022-03-27 09:51:38.120286
128
2022-03-27 09:52:17.978380
256
2022-03-27 09:53:32.402208
512
2022-03-27 09:55:56.901353
1024
2022-03-27 10:01:06.021471
2048
2022-03-27 10:11:44.900135
4096
Finished:  2022-03-27 10:16:04.768877  Errors: 804839
2022-03-27 10:16:04.843661
1
2022-03-27 10:16:05.113522
2
2022-03-27 10:16:05.665779
4
2022-03-27 10:1

In [14]:
#with open('ISM_Training.dataframe_1', 'wb') as f:
#    pickle.dump(ISM_Training, f, protocol=pickle.HIGHEST_PROTOCOL)
#with open('ISM_Training.dataframe_1', 'rb') as f:
#    ISM_Training_load = pickle.load(f)


In [94]:
#Integrar set de training_data y ajustar el indice al TICKETID

#ISM_Training_load = ISM_Training_1
ISM_Training_load = ISM_Training_1.append(ISM_Training_2)
ISM_Training_load = ISM_Training_load.append(ISM_Training_3)
#ISM_Training_load = ISM_Training_load.append(ISM_Training_4)

with open('ISM_Training_load.dataframe_1', 'wb') as f:
    pickle.dump(ISM_Training_load, f, protocol=pickle.HIGHEST_PROTOCOL)

#ISM_Training_load = ISM_Training_load.set_index('TICKETID') 
ISM_Training_load.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 14447 entries, 0 to 4448
Columns: 164 entries, I-CEX-IN-ECS-O365-L3 to TICKETID
dtypes: object(164)
memory usage: 18.2+ MB


In [48]:
#with open('ISM_Training_load.dataframe', 'wb') as f:
#    pickle.dump(ISM_Training_load, f, protocol=pickle.HIGHEST_PROTOCOL)


#with open('ISM_Training_load.dataframe', 'rb') as f:
#    ISM_Training_load = pickle.load(f)

In [69]:
#ISM_Training_load = ISM_Training_load.reset_index()
#ISM_Training_load.info()

In [51]:
#count = 0
#for idx in ISM_Training_load.index:
#    count = count + 1
#    if count == 500:
#        print(count)
#for column in ISM_Training_load.columns:
#    if column not in [0,'COUNTRY','OWNERGROUP','TICKETID']:
    #value = int(ISM_Training_load[column][idx])
#        ISM_Training_load[column] = ISM_Training_load[column]**2


In [95]:

ISM_Training_load

,I-CEX-IN-ECS-O365-L3,I-CEX-MX-IT-SEC-DTC-L2,I-CEX-IN-IT-IAM-DTC-L2,I-CEX-MX-COM-LE-GSC-L1,I-CEX-MXBOSAP-SEC-GRC-L3,I-CEX-US-IBMSECURITYUSA-L3,I-CEX-EMEAA-BOCOM-GSC-L1,I-CEX-GLOBAL-IT-CNX-L2,I-CEX-GLOBAL-IT-LNT-GSC-L1,I-CEX-GLOBAL-IT-CNX-MON-L2,...,I-CEX-US-DSS-WPB-L2,I-CEX-MX-HR-SER-PR-L2,I-CEX-MX-CONTACTCENTER-GV-L2,C-CEX-MX-CONTACTCENTER-GV-L2,I-CEX-GLOBAL-IT-MSSSOC-INTQRAD-DTC-L2,I-CEX-MX-IT-SEC-CAB,I-CEX-MX-IT-SCCM-APP-L2,COUNTRY,OWNERGROUP,TICKETID
0,841,900,1369,1089,1681,784,441,841,841,9,...,1,1,1,1,0,0,0,Mexico,I-CEX-MXBOSAP-SEC-APP-L3,SR2677812
1,7569,26244,30625,28224,6400,28224,25921,24964,5776,9,...,1,16,4,16,16,4,1,Mexico,C-CEX-MX-EA-BASIS-PR-CX-L3,SR2550470
2,25,16,36,36,16,4,4,16,25,0,...,0,1,0,4,0,0,0,Mexico,I-CEX-NOAM-IT-GSC-L1,SR2670499
3,0,0,1,9,0,0,0,1,1,0,...,0,0,0,0,0,0,0,Mexico,I-CEX-MX-OP-GINCO-GSC-L1,SR2672145
4,225,361,529,441,196,64,16,289,289,1,...,0,25,0,0,0,0,1,Mexico,I-CEX-MX-IT-SER-OS-L2,SR2668995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4444,961,256,1089,1156,289,256,144,225,784,1,...,4,1,1,1,4,1,0,Mexico,I-CEX-LATAM-AFST-TAX-PR-MNT-L3,SR2645967
4445,100,49,400,529,169,64,25,225,289,1,...,0,16,1,1,0,0,0,Mexico,I-CEX-MX-BO-JDE-GSC-L1,SR2643236
4446,49,36,324,64,25,36,25,16,361,1,...,0,9,0,0,0,0,0,Mexico,I-CEX-GLOBAL-ITBDCIAMAP-GSC-L1,SR2663588
4447,25,25,64,49,9,4,0,49,64,0,...,0,1,0,1,0,0,0,Mexico,I-CEX-NOAM-IT-GSC-L1,SR2663076


## Start Pipeline

In [96]:
## Create Y Data / Transform Y Data
ISM_Training_load_1 = ISM_Training_load

## Clase Prepare Data
#    Instance Object()
#    Fit(ISM_Trainind_Data)
#    X_train, X_test, y_train, y_test = Transform()




In [97]:
##Convertimos los grupos en clasificaicones numéricas

ownergroup_mapping = pd.DataFrame(ISM_Training_load['OWNERGROUP'].unique(), columns=['OWNERGROUP'])
ownergroup_mapping['MAP_VALUE'] = ownergroup_mapping.index
ownergroup_mapping.head(3)

,OWNERGROUP,MAP_VALUE
0,I-CEX-MXBOSAP-SEC-APP-L3,0
1,C-CEX-MX-EA-BASIS-PR-CX-L3,1
2,I-CEX-NOAM-IT-GSC-L1,2


In [120]:
#ISM_Training_load['INDEX']=ISM_Training_load.index
ISM_Training_load_full = pd.merge(ISM_Training_load, ownergroup_mapping, on='OWNERGROUP')
ISM_Training_load_full.head(3)

## Construimos X y Y

X_Train_Data = ISM_Training_load_full.drop(['OWNERGROUP', 'MAP_VALUE'], axis=1)
Y_Train_Data = ISM_Training_load_full[['TICKETID','MAP_VALUE']]
Y_Train_Data = Y_Train_Data.set_index('TICKETID') 

Y_Train_Data.head(5)

##Split Trainng Data 
## Tenemos que hacer el encoder antes, por el tema de los países, pueden salir columnas diferentes
## Al productizar, tenemos que creear una matriz de países y que sa la estructura base e ignorar el resto.

datatrans_encoder = ColumnTransformer([
    ('one_hot', OneHotEncoder(handle_unknown='ignore'), ['COUNTRY'])
], remainder='passthrough', verbose_feature_names_out=False)

df_temp = datatrans_encoder.fit_transform(X_Train_Data)
df_temp = pd.DataFrame(df_temp)
df_temp.columns = datatrans_encoder.get_feature_names_out()

X_train_Data_fixed = df_temp
X_train_Data_fixed = X_train_Data_fixed.set_index('TICKETID') 
X_train_Data_fixed.head(5)

## Hacemos Split de Datos

X_train, X_test, y_train, y_test = train_test_split(X_train_Data_fixed, Y_Train_Data, test_size=0.33)

## Cambio tipo de datos a númerico para procesos de training de modelos
X_train_fixed = X_train.apply(pd.to_numeric)
X_test_fixed = X_test.apply(pd.to_numeric)


X_train_fixed.head(5)

,COUNTRY_Mexico,I-CEX-IN-ECS-O365-L3,I-CEX-MX-IT-SEC-DTC-L2,I-CEX-IN-IT-IAM-DTC-L2,I-CEX-MX-COM-LE-GSC-L1,I-CEX-MXBOSAP-SEC-GRC-L3,I-CEX-US-IBMSECURITYUSA-L3,I-CEX-EMEAA-BOCOM-GSC-L1,I-CEX-GLOBAL-IT-CNX-L2,I-CEX-GLOBAL-IT-LNT-GSC-L1,...,I-CEX-US-FIELD-TECHS,I-CEX-MX-HR-SER-SSC-MNT-L1,I-CEX-US-DSS-HOUSTON-L2,I-CEX-US-DSS-WPB-L2,I-CEX-MX-HR-SER-PR-L2,I-CEX-MX-CONTACTCENTER-GV-L2,C-CEX-MX-CONTACTCENTER-GV-L2,I-CEX-GLOBAL-IT-MSSSOC-INTQRAD-DTC-L2,I-CEX-MX-IT-SEC-CAB,I-CEX-MX-IT-SCCM-APP-L2
TICKETID,,,,,,,,,,,,,,,,,,,,,
SR2655964,1.0,121,81,400,225,49,16,9,49,196,...,1,9,4,1,4,0,1,0,0,4
SR2681716,1.0,4,4,9,4,4,1,1,4,4,...,0,16,0,0,4,0,0,0,0,0
SR2678251,1.0,9,9,9,25,9,4,4,16,9,...,0,100,0,0,16,1,1,0,0,0
SR2668535,1.0,169,49,225,676,81,36,16,256,196,...,0,1,0,0,25,0,1,0,0,0
SR2653344,1.0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,4,0,0,0,0,0


## Base Models

In [101]:
### Dummy Model
dum_model = DummyClassifier(strategy='uniform').fit(X_train_fixed, y_train)

In [102]:
yhat_dc = dum_model.predict(X_train_fixed)
accuracy_score(y_train, yhat_dc), precision_score(y_train, yhat_dc, average='macro'), recall_score(y_train, yhat_dc, average='macro')

(0.007438784998450253, 0.007533940577188222, 0.02314671766955725)

In [ ]:
## Baseline
## Objective: Reduce at least 50% time of agetn assignment time
## Average assignment rate 3 min / Ticket = 10 tickets = 30 mins
##                            50%  Ticket = 5 * 1 + 5 * 3 = 22.5 = 66% (Saved time)
##                            60%  Ticket = 6 * 1 + 4 * 3 = 21.5 = 60%
##                            70%  Ticket = 7 * 1 + 3 * 3 = 19.5 = 53%
##                            80%  Ticket = 8 * 1 + 2 * 3 = 18.0 = 46%
##                            90%  Ticket = 9 * 1 + 1 * 3 = 16.5 = 40%


In [103]:
predictmodel = xgb.XGBClassifier()
predictmodel.fit(X_train_fixed,y_train)

C:\Users\mamoralessr\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = 

[13:41:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [104]:
y_train_predicted = predictmodel.predict(X_train_fixed)

In [39]:
#y_train_predicted

In [105]:
## (0.988050784167289, 0.8999303353339181, 0.9141716566866268)
## (0.9917849141150112, 0.9316936463693539, 0.9420673076923076) - 3 dataset para pabalar 1 daata set de training
## (0.9940253920836445, 0.9401629426474146, 0.9503105590062112) - 3 dataset para pabalar + optimización de palabras , 1 daata set de training 
#                                                                 / optimización performaance training data
## (0.9977595220313666, 0.9498606141920094, 0.958139534883721) - 3 dataset para palabras y training - mejora de palabras
## (1.0, 1.0, 1.0)   - solo datos de méxico, la distribucóón de palabras incorrecta
## (1.0, 1.0, 1.0)   - solo datos de méxico (1-set), la distribucóón de palabras sigue incorrecta
## (0.9970104633781763, 0.9652777777777778, 0.9722222222222222) - Solo 1000 registros mexico, mejora depuración regex, eliminar repeticiones  2
## (0.9970104633781763, 0.9670220326936746, 0.9701492537313433) - Solo 1000 registros mexico, mejora depuración regex, eliminar repeticiones  2 - eliminar nombres
## (0.9996292176492398, 0.999947254958034, 0.9998651979956652) - Todo Mexico (8056) -  mejora depuración regex, eliminar repeticiones  2 - eliminar nombres
## (0.996292176492399, 0.9893615163073012, 0.9871562449500731) - Anterior + el cuadrado de match de palabras
## (0.9994438264738599, 0.9797980294920234, 0.9841933139534884) - El proceso anteiorr + curadro de los valores sobre la tabal de trainings
## (0.9948341770844096, 0.9977036850547525, 0.9924358040679491) - Agreagar un mes completo de datos para training
accuracy_score(y_train, y_train_predicted), precision_score(y_train, y_train_predicted, average='macro'), recall_score(y_train, y_train_predicted, average='macro')

(0.9948341770844096, 0.9977036850547525, 0.9924358040679491)

In [106]:
y_test_predicted = predictmodel.predict(X_test_fixed)

In [107]:
## (0.7333333333333333, 0.4699628058733169, 0.45581237966684646)
## (0.6348484848484849, 0.3386554337862025, 0.3552215795132643) 3 dataset para pabalar 1 daata set de training
## (0.6166666666666667, 0.31582311366794125, 0.33113032812469756) 3 dataset para pabalar + optimización de palabras , 1 daata set de training 
#                                                                  / optimización performaance training data
## (0.7434343434343434, 0.509846497720536, 0.4785910693035512)  3 dataset para palabras y training - mejora de palabras
## (0.7603461249059443, 0.48218784214388655, 0.42512328682302225) - solo datos de méxico, la distribucóón de palabras incorrecta
## (0.7348484848484849, 0.446154673658955, 0.4402168010724709) - solo datos de méxico 1-set, la distribucóón de palabras incorrecta
## (0.7045454545454546, 0.3638892520212744, 0.3594193305460011) - solo datos de méxico 1-set, la distribucóón de palabras icorregida
## (0.7242424242424242, 0.4070510067010857, 0.3896439704587102) - Solo 1000 registros mexico, mejora depuración regex, eliminar repeticiones  2
## (0.6787878787878788, 0.36347548907295174, 0.37880837133970213)- Solo 1000 registros mexico, mejora depuración regex, eliminar repeticiones  2 - eliminar nombres
## (0.7994732881866065, 0.5708462677095836, 0.49161033884617494) - Todo Mexico (8056) -  mejora depuración regex, eliminar repeticiones  2 - eliminar nombres
## (0.7968397291196389, 0.5515521425067268, 0.48502100207879784) - MIsmo que anterior.. (porque hay diferencia??)
## (0.7990970654627539, 0.5824423370084997, 0.5295225892375374) - Anterior + el cuadrado de match de palabras
## (0.8116610738255033, 0.6460636633552914, 0.5525619906042211) - Magreagar un mes completo de datos para training - solo cuadrado de math de palabras
accuracy_score(y_test, y_test_predicted), precision_score(y_test, y_test_predicted, average='macro'), recall_score(y_test, y_test_predicted, average='macro')


C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.8116610738255033, 0.6460636633552914, 0.5525619906042211)

## Grid Search

In [109]:

gs = GridSearchCV(predictmodel, {'n_estimators':range (1, 10, 100), 'max_depth':range (1, 50, 100)}, cv=5, n_jobs=-1, verbose=0, scoring = 'accuracy')
gs.fit(X_train_fixed, y_train)
gs.best_score_ , gs.best_params_

C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\mamoralessr\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\mamoralessr\Anaconda3\lib\site-packages

[13:46:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


(0.3474537126017041, {'max_depth': 1, 'n_estimators': 1})

In [110]:
predictmodel_final = xgb.XGBClassifier({'max_depth': 1, 'n_estimators': 1})
predictmodel_final.fit(X_train_fixed,y_train)

[13:46:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [114]:
parameters = {'n_estimators': range(100, 200, 25),
              'criterion': ['gini', 'entropy'],
              'max_depth': list(range(1, 5)) + [None],
              'max_features': ['auto', 'sqrt', 'log2']
             }
model = RandomForestClassifier()
gs = GridSearchCV(model, parameters, cv=5, n_jobs=-1, verbose=1)
gs.fit(X_train_fixed, y_train)
gs.best_score_, gs.best_params_

Fitting 5 folds for each of 120 candidates, totalling 600 fits


C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:910: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


(0.7332364341085271,
 {'criterion': 'gini',
  'max_depth': None,
  'max_features': 'auto',
  'n_estimators': 175})

In [117]:
Forestmodel = RandomForestClassifier(**gs.best_params_)
Forestmodel.fit(X_train_fixed, y_train)



C:\Users\MAMORA~1\AppData\Local\Temp/ipykernel_36892/4239747531.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  Forestmodel.fit(X_train_fixed, y_train)


RandomForestClassifier(max_features='auto', n_estimators=175)

In [121]:
y_test_predicted = Forestmodel.predict(X_test_fixed)
accuracy_score(y_test, y_test_predicted), precision_score(y_test, y_test_predicted, average='macro'), recall_score(y_test, y_test_predicted, average='macro')

C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.9058305369127517, 0.8843810833150799, 0.7534218420171158)

## Final Model

In [111]:
##1- (0.7333333333333333, 0.4699628058733169, 0.45581237966684646)
##2- (0.6348484848484849, 0.3386554337862025, 0.3552215795132643)
##3- (0.6166666666666667, 0.31582311366794125, 0.33113032812469756)
##4- (0.7434343434343434, 0.509846497720536, 0.4785910693035512) - 3 dataset para palabras y training - mejora de palabras
##5- (0.7603461249059443, 0.48218784214388655, 0.42512328682302225) - solo datos de méxico, la distribucóón de palabras incorrecta
##6- (0.7348484848484849, 0.446154673658955, 0.4402168010724709) - solo datos de méxico 1-set, la distribucóón de palabras incorrecta
##7- (0.7045454545454546, 0.3638892520212744, 0.3594193305460011) - solo datos de méxico 1-set, la distribucóón de palabras icorregida
##8- (0.7242424242424242, 0.4070510067010857, 0.3896439704587102) - Solo 1000 registros mexico, mejora depuración regex, eliminar repeticiones  2
##9- (0.6787878787878788, 0.36347548907295174, 0.37880837133970213)- Solo 1000 registros mexico, mejora depuración regex, eliminar repeticiones  2 - eliminar nombres
##10-(0.7994732881866065, 0.5708462677095836, 0.49161033884617494) - Todo Mexico (8056) -  mejora depuración regex, eliminar repeticiones  2 - eliminar nombres
##   (0.7968397291196389, 0.5515521425067268, 0.48502100207879784) - MIsmo que anterior.. (porque hay diferencia??) - Solo se hizo limpieza de código y agregaar index
##11-(0.7990970654627539, 0.5824423370084997, 0.5295225892375374) - Mismo que anterior + cuadrado de match de palabras.. accuracy ligeramente mejor y mejora principalment en precision y recall
##11A (0.7990970654627539, 0.5419410058175115, 0.5031069709866498) - Se utilizó el cuadrado sobre la tabla de training directamente, resultó mejor el resultado de cuadrados sobre la match de palabras.
##12 (0.8116610738255033, 0.6460636633552914, 0.5525619906042211) - Magreagar un mes completo de datos para training - solo cuadrado de math de palabras
y_test_predicted = predictmodel.predict(X_test_fixed)
accuracy_score(y_test, y_test_predicted), precision_score(y_test, y_test_predicted, average='macro'), recall_score(y_test, y_test_predicted, average='macro')
##13 (0.9058305369127517, 0.8843810833150799, 0.7534218420171158) - Al utilzar el random forest en lugar de XGBoost obtuvimos un valor mejor.


C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mamoralessr\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.8116610738255033, 0.6460636633552914, 0.5525619906042211)

In [139]:
## Conclusiones
## Los parametros proporcioandos por el grid search no nosmejoran la predicción,
## Debemos buscarle en la seleccción de las palabras

## Entrenar con el mismo set con el que se generan los set de palabras mejora sustancialmente el resultado
## Reducir el alcance a México y mejorar el filtrado de palabras debe subir a menos un 10%

## Corridas 6 y 7
## Al parecer el usar un solo set (2000 registros) si impacta negativamente.. adicionalmente la limpieza de palabras parece tener un impacto negativo en ese punto
## Siguietne Acció: Utilizar el set completo de México / y Analizar el set de palabras y exlusiones

## Corrida 9
## Eliminar los nombres en el modelo a 1000 registros resultó negativo
## Corrida 10
## Confirmamos mejor resultado.. El número de registros de de training sumado a la eliminación de nombres impacta positivamente en la predicción
##   Siguiente paso: 
##      Ajustar el proceso con el index = ticketid para poder analyzar los registros fallidos
##      Analizar los registros fallidos, existe alguna relación entre estos grupos?
##           Revisando los casos fallidos, vemos el el valor lógico si conside con el valor Real llevando el % de efectividad a 95%
##           Habilitar un exponenciador del resultod (valores cuadrados por ejemplo) debería ayudar a las predicciones
## Corrida 11
##   utilizando ls cuadrados del match de palabras, se mejora principalmente el recall y precision.. 
##      ? Hipotesis: Si aplicamos el cuadrado a la suma total, mejorará el resultado?
##      Incrementar el número de registros para set de trainnings (2 meses)
##      Darle peso excesivo a las palabras que hacen match con el group name        
## Corrida 11A
##   Se elevaron al cuadrado los valors de la tabal de training anterior, cayeron los valores resultó mejor el resultado de cuadrados sobre la match de palabras.
## Corrida 12
##  Se entrenó con un mes completo de información, logramo incremento de .012% de mejor.
##  --Aún falta idnetificar que modelo podría considerar mejor los dataos que las fallas, que en la matoría e valor mayor es el correcto.
##  --Podmeos hacer tambien un mo delo que evalúe directamente sobre el valor mayor.
## Corrida 13
##  -- Utilizando el modelo RandomForest se obtiene in 90%, las caracteristicas de este Modelo favorecen a la estructura de nuestra información


In [122]:
## Preapramos información para revisión y analisis de errores.

ownergroup_mapping
groups_dic = {ownergroup_mapping.loc[idx, 'MAP_VALUE']: ownergroup_mapping.loc[idx, 'OWNERGROUP'] for idx in range(ownergroup_mapping.shape[0])}
X_test_fixed['PREDICT'] = y_test_predicted
X_test_fixed['REAL'] = y_test
X_test_fixed['PREDICT'] = X_test_fixed['PREDICT'].map(groups_dic)
X_test_fixed['REAL'] = X_test_fixed['REAL'].map(groups_dic)
X_test_fixed[['PREDICT','REAL']].head(3)




,PREDICT,REAL
TICKETID,,
SR2651035,I-CEX-MX-COM-LE-GSC-L1,I-CEX-MX-OP-SAP-RDM-APP-L3
SR2670267,I-CEX-MX-OP-SAP-PROC-L3,I-CEX-MX-OP-SAP-PROC-L3
SR2674081,I-CEX-NOAM-IT-GSC-L1,I-CEX-NOAM-IT-GSC-L1


In [123]:
X_test_fixed.to_excel(f"./Data/ResultAnalysis_Forest.xlsx")



In [37]:
X_test_fixed

,COUNTRY_Mexico,C-CEX-DE-IT-SAP-SEC-L2,I-CEX-US-IBMSECURITYUSA-L3,I-CEX-MX-IT-SEC-DTC-L2,I-CEX-IN-IT-IAM-DTC-L2,I-CEX-CZ-IT-GSC-L1,I-CEX-MX-COM-LE-GSC-L1,I-CEX-GLOBAL-ITBDCIAMAP-GSC-L1,I-CEX-GLOBAL-IT-LNT-GSC-L1,I-CEX-IN-ECS-O365-L3,...,I-CEX-MX-COM-MM-APP-L3,I-CEX-IT-SELFSERVPORTAL-VIDEOCONF-L2,C-CEX-CO-IT-INM-ROS-OCC-L2,C-CEX-MX-CONTACTCENTER-GV-L2,I-CEX-MX-BO-APP-GSC-L1,I-CEX-IN-IT-OAM-DTC-L2,I-CEX-US-APPS-S&OP,I-CEX-IN-IT-BURA-DTC-L2,PREDICT,REAL
TICKETID,,,,,,,,,,,,,,,,,,,,,
SR2643316,1.0,2,3,2,11,1,14,12,11,6,...,3,1,1,0,3,0,0,0,I-CEX-MX-PSD-LE-PA-L3,I-CEX-MX-PSD-LE-PA-L3
SR2653162,1.0,27,26,19,39,16,37,40,20,36,...,1,2,1,0,1,3,0,0,I-CEX-LATAM-AFST-TAX-PR-MNT-L3,I-CEX-IN-IT-CITRIX-DTC-L2
SR2667239,1.0,1,1,5,5,0,6,4,3,4,...,0,0,0,0,2,0,0,0,I-CEX-NOAM-IT-GSC-L1,I-CEX-NOAM-IT-GSC-L1
SR2641967,1.0,7,13,10,25,3,35,30,26,18,...,3,1,4,2,2,2,1,0,I-CEX-MX-COM-BW-APP-L3,I-CEX-MX-COM-BW-APP-L3
SR2655022,1.0,3,3,3,5,1,7,4,5,4,...,1,1,1,0,1,0,0,0,I-CEX-MX-COM-SAP-SL-APP-L3,I-CEX-MX-COM-SAP-SL-APP-L3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SR2647020,1.0,2,4,4,9,1,14,10,8,8,...,2,1,1,1,1,1,0,0,I-CEX-MX-COM-SAP-SL-APP-L3,I-CEX-MX-COM-SAP-SL-APP-L3
SR2663508,1.0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,I-CEX-MX-COM-LE-GSC-L1,I-CEX-MX-COM-LE-GSC-L1
SR2665484,1.0,3,4,6,9,0,19,7,9,6,...,0,0,1,0,1,0,0,0,I-CEX-GLOBAL-IT-DYN-L2,I-CEX-GLOBAL-IT-DYN-L2


## Test NameDataset

In [12]:
nd = NameDataset()

In [15]:
dict_names = nd.search('##CUCARAMACARATITIRIFUE##').get('first_name')


In [16]:
print(dict_names)

None


In [25]:
#print(nd.search('STEVEN').get('first_name').get('country').get('Mexico'))
errcount = 0
try:
    is_name = nd.search('MARIO').get('first_name').get('country').get('Mexico')
    if is_name == 'None':
        print('graba')
except:
    errcount = errcount + 1
    print('graba1')
#else:
#    errcount = errcount + 1
        
#print(nd.search('CUCARAMACARATITIRIFUE').get('first_name').get('country').get('Mexico'))

In [99]:
type(x)

NoneType

In [103]:
if nd.search('mistico').get('first_name') == x :
    print('None-1')
else:
    print(nd.search('TELEFONO'))



{'first_name': None, 'last_name': None}


## Test 1

In [50]:
count = 0
count2 = 0
count_check = 1


for numindex, ind in enumerate(ISM_test.index):
    count = count + 1
    if count == count_check:
        print(datetime.now())
        print(count)
        
        if count == count_check:
            count_check = count_check + count_check
        
    cleanstring = ISM_test['FULLDESC'][ind].replace("\\N"," ")
    cleanstring = cleanstring.replace(r"\N"," ")
    word_list = cleanstring.split(" ")
    
    
    #TrainingData = TrainingData.append({'COUNTRY':ISM_test['AffectedPersonCountry'][ind],'OWNERGROUP':ISM_test['OWNERGROUP'][ind]}, index=[ind]) 
    #ISM_wgwords = ISM_wgwords.append({'COUNTRY':ISM_test['AffectedPersonCountry'][ind],'OWNERGROUP':ISM_test['OWNERGROUP'][ind],'WORDS':word.upper()}, ignore_index=True)
    
    TrainingData = TrainingData.append([ISM_test['AffectedPersonCountry'][ind],ISM_test['OWNERGROUP'][ind]], ['COUNTRY','OWNERGROUP'],[numindex])
    #TrainingData = TrainingData.append({'COUNTRY':ISM_test['AffectedPersonCountry'][ind],'OWNERGROUP':ISM_test['OWNERGROUP'][ind]}, ignore_index=True) 
    
    for column in TrainingData.columns:
        if column not in ['0','COUNTRY','OWNERGROUP']:
            TrainingData[column][ind] = 0
            
    
    #print(ISM_test['AffectedPersonCountry'][ind])
    #print(TrainingData['COUNTRY'][ind])
    
    listwords = ISM_wgwords_uniques[(ISM_wgwords_uniques['COUNTRY']==ISM_test['AffectedPersonCountry'][ind])][['WORDS','OWNERGROUP']]
    #print(len(listwords))
    
    for word in word_list:
        if len(word) > 2:
                if word.upper() not in ['[Screenshot','THE','AND','ATTACHMENT','PLEASE','FOR','CEMEX','SUBJECT','FAVOR','RE:','CAUTION','REGARDS','SALUDOS','listener.]','FROM:','SUBJECT:','CC:','TO:','FW:',\
                                         'SENT:','GSC', 'THIS', 'DEL','POR','PARA','QUE','CON'] \
                and word.find('cid:') == -1 and word.find('cid:') == -1  and word.find('@cemex.com') == -1 and word.find('@ext.cemex.com') == -1 and word.find('image') == -1 and word.find('>>;') == -1 \
                and word.find('GSC') == -1 and word.find('______') == -1 and word.find('---') == -1 and word.find('  ') == -1 and word.find('.gif') == -1 and word.find('safelinks') == -1 \
                and word.find('.png') == -1 and word.find('E0') == -1 and word.find('E-') == -1 and word.find('****') == -1 and word.find('#') == -1 and word.find('listener.]') and word.find('you') and word.find('Regards') \
                and word.find('del') and word.find('para') and word.find('por') and word.find('saved') and word.find('was') and word.find('are') and word.find('del') \
                and word.find('e-mail') and word.find('este') and word.find('[Screenshot') and word.find('las') and word.find('los') and word.find('not') and word.find('Thank')\
                and not word.startswith('+') and not word[0].isdigit() and not word[1].isdigit() and not word.startswith('<') and not word.startswith('/>') and not word.startswith('@'):
                    
                    #grouplist = ISM_wgwords_uniques[ISM_wgwords_uniques['COUNTRY']==ISM_test['AffectedPersonCountry'][ind]][['OWNERGROUP']]
                    #for group in grouplist['OWNERGROUP']:
                    #    print(group)
                    #    #listwords = ISM_wgwords_uniques[ISM_wgwords_uniques['OWNERGROUP']==group & ISM_wgwords_uniques['COUNTRY']==ISM_test['AffectedPersonCountry'][ind]][['WORDS']]
                    #    listwords = ISM_wgwords_uniques[(ISM_wgwords_uniques['OWNERGROUP']==group)&(ISM_wgwords_uniques['COUNTRY']==ISM_test['AffectedPersonCountry'][ind])][['WORDS']]
                    #    if word in listwords['WORDS']:
                    #        print(group)
                    #        TrainingData[group][ind] = TrainingData[group][ind] + 1
                    #print('Valid Word')
                    #print(word)
                                        
                    #for idx1 in listwords:
                    #    print(idx1)
                        #print(listwords['WORDS'][idx1])
                    #if word == listwords['WORDS'][idx1]:
                    #print(group)
                    #    TrainingData[listwords['OWNERGROUP']][ind] = TrainingData[listwords['OWNERGROUP']][ind] + 1
                    
                    for listidx in listwords.index:
                        if listwords['WORDS'][listidx] == word :
                            #print(listwords['OWNERGROUP'][listidx])
                            #print("Word Found")
                            #print('TrainingData[',listwords['OWNERGROUP'][listidx],'][',ind,']','= TrainingData[',listwords['OWNERGROUP'][listidx],'][',ind,'] + 1')
                            #print(TrainingData[listwords['OWNERGROUP'][listidx]][ind])
                            #print(numindex)
                            TrainingData[listwords['OWNERGROUP'][listidx]][numindex] = TrainingData[listwords['OWNERGROUP'][listidx]][numindex] + 1
                            #print(TrainingData[listwords['OWNERGROUP'][listidx]][ind])

TrainingData.info()


2022-03-21 13:43:52.479470
1
2022-03-21 13:43:52.695937
2
2022-03-21 13:43:59.035910
4
2022-03-21 13:44:05.338098
8
2022-03-21 13:44:21.304474
16
2022-03-21 13:44:39.254132
32
2022-03-21 13:45:41.697980
64
2022-03-21 13:46:31.151217
128
2022-03-21 13:49:52.843760
256


KeyboardInterrupt: 

In [542]:
TrainingData.describe()
TrainingData[['COUNTRY','OWNERGROUP','I-CEX-MX-OP-GINCO-GSC-L1','I-CEX-NOAM-IT-GSC-L1']]

,COUNTRY,OWNERGROUP,I-CEX-MX-OP-GINCO-GSC-L1,I-CEX-NOAM-IT-GSC-L1
0,Colombia,I-CEX-MX-OP-GINCO-GSC-L1,10,4
1,GB,C-CEX-FR-BSO-FA-PR-L2,0,0
2,India,I-CEX-EMEA-IT-SERVICE DESK-L1,0,0
3,Mexico,I-CEX-NOAM-IT-GSC-L1,3,5
4,Mexico,I-CEX-NOAM-IT-GSC-L1,5,11
5,Spain,I-CEX-ES-IT-INM-ROS-VLC-L2,0,4
6,Colombia,I-CEX-NOAM-IT-GSC-L1,30,158


## Prueba SET

In [114]:
#cleanstring = ISM_test['FULLDESC'][1].replace("\\N"," ")
#cleanstring = cleanstring.replace(r"\N"," ")
#word_list = cleanstring.split(" ")
#word_list



In [115]:
#grouplistwords = ISM_wgwords_uniques[(ISM_wgwords_uniques['COUNTRY']==ISM_test['AffectedPersonCountry'][1]) &(ISM_wgwords_uniques['OWNERGROUP']==ISM_test['OWNERGROUP'][1]) ][['WORDS']]
#xxlist = grouplistwords['WORDS'].to_list()
#yylist = set(word_list) & set(xxlist)
#yylist

In [28]:

GroupColumnsNames =ISM_wgwords['OWNERGROUP'].unique()
TrainingData = pd.DataFrame(columns=GroupColumnsNames)
TrainingData['COUNTRY'] = 'NA'
TrainingData['OWNERGROUP'] = 'NA'

TrainingData.info()
TrainingData.COUNTRY.datatype = object
TrainingData.OWNERGROUP.datatype = object

count = 0
count2 = 0
count_check = 1


for numindex, ind in enumerate(ISM_test.index):
    count = count + 1
    if count == count_check:
        print(datetime.now())
        print(count)
        
        if count == count_check:
            count_check = count_check + count_check
            #if count_check == 32:
            #    break
            
    cleanstring = ISM_test['FULLDESC'][ind].replace("\\N"," ")
    cleanstring = cleanstring.replace(r"\N"," ")
    word_list = cleanstring.split(" ")
    
    TrainingData = TrainingData.append({'COUNTRY':ISM_test['AffectedPersonCountry'][ind],'OWNERGROUP':ISM_test['OWNERGROUP'][ind]}, ignore_index=True) 
    #TrainingData = TrainingData.append([ISM_test['AffectedPersonCountry'][ind],ISM_test['OWNERGROUP'][ind]], ['COUNTRY','OWNERGROUP'],[numindex])
    
    ## Seteo valores de columnas de grupos del registros nuevo para que no sean null y puedan sumarse
    for column in TrainingData.columns:
        if column not in [0,'COUNTRY','OWNERGROUP']:
            TrainingData[column][numindex] = 0
    
    for columns in GroupColumnsNames:
        grouplistwords = ISM_wgwords_uniques[(ISM_wgwords_uniques['COUNTRY']==ISM_test['AffectedPersonCountry'][ind]) &(ISM_wgwords_uniques['OWNERGROUP']==columns) ][['WORDS']]
        grouplistwords_list = grouplistwords['WORDS'].to_list()
        
        ## el & de lista busca obtiene lso valores que hacen match
        list_matches = set(word_list) & set(grouplistwords_list)
        if len(list_matches) > 0:
            TrainingData[columns][numindex] = TrainingData[columns][numindex] + len(list_matches)


TrainingData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Columns: 183 entries, I-CEX-MX-BO-JDE-GSC-L1 to OWNERGROUP
dtypes: object(183)
memory usage: 0.0+ bytes
2022-03-21 15:53:48.799973
1
2022-03-21 15:53:49.619435
2
2022-03-21 15:53:51.418975
4


KeyboardInterrupt: 

In [30]:
TrainingData

,I-CEX-MX-BO-JDE-GSC-L1,I-CEX-MX-OP-GINCO-GSC-L1,I-CEX-MXBOSAP-SEC-APP-L3,I-CEX-MX-COM-LE-GSC-L1,I-CEX-NOAM-IT-GSC-L1,I-CEX-MX-PSD-LE-PA-L3,I-CEX-GLOBAL-IT-CNX-L2,I-CEX-GLOBAL-IT-LNT-GSC-L1,I-CEX-MX-IT-SER-RS-L2,I-CEX-ES-IT-INM-ROS-VLC-L2,...,I-CEX-US-COMMS-L3,C-CEX-MX-IT-GINCOSOL-APP-L2,I-CEX-IN-IT-MAILCOL-DTC-L2,I-CEX-MX-HR-SER-PR-L2,C-CEX-GLOBAL-IT-AXIANS-DTC-L2,V-CEX-AT&T-WAN,C-CEX-HR-IT-CP-L2,I-CEX-US-APPS-S&OP,COUNTRY,OWNERGROUP
0,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,United Kingdom,C-CEX-FR-BSO-FA-PR-L2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Jamaica,I-CEX-US-SERVICEDESK-L1
2,23,12,14,17,18,9,4,16,4,0,...,0,1,0,0,0,0,0,0,Mexico,C-CEX-DE-IT-SAP-SEC-L2
3,7,0,0,4,5,0,0,0,0,5,...,0,0,0,0,0,0,0,0,Spain,I-CEX-NOAM-IT-GSC-L1
4,18,17,11,17,17,12,9,15,6,0,...,0,1,0,2,0,0,0,0,Mexico,I-CEX-MXBOSAP-PRCL3
5,16,24,13,41,14,21,14,27,29,0,...,0,0,0,1,0,0,0,0,Mexico,I-CEX-MX-COM-SAP-SL-APP-L3
6,2,2,0,6,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,Mexico,I-CEX-MX-COM-LE-GSC-L1


## Word CLeaning testing

In [79]:
import unidecode
import string
import re


wdlist = ['uno', 'UNO', '  UNÓ', 'Uno<<', 'unó:[]', 'uno: ']
wdlist = [re.sub('[?:"><\'. ()!¡\[\]]', '',unidecode.unidecode(wd).upper()) for wd in wdlist]
#new_string = a_string.translate(str.maketrans('', '', string.punctuation))
wdlist = list(dict.fromkeys(wdlist))
wdlist

['UNO']

In [65]:
for wd in wdlist: 
    print()

uno
UNO
UNÓ
Uno
unó:
uno:


In [73]:
import re

word = "  esto esto:?  "
word = re.sub('[?:"><\'.()!¡]', '',word)
word

'  esto esto  '

In [ ]:
# import required module
import unidecode
 
# assign string
string = "orčpžsíáýd"
 
# display original string
print('\nOriginal String:', string)
 
# remove ascents
outputString = unidecode.unidecode(string)
 
# display new string
print('\nNew String:', outputString)